# Iteration 1: Acute Stroke Unit Arrivals stage 2

## Plain English Summary

The first step in coding a `simpy` model is typically coding the arrivals to the system.  In the Acute Stroke Unit (ASU) part of the model accepts four patient type arrivals model: strokes, transient ischemic attacks (TIA; these are strokes where symptoms resolve rapidly), complex neurological, and other (medical outliers).



## LLM Prompt

::::{admonition} LLM Prompt
:::{include} ../04_prompts_stage1/inputs/01_iteration.md
:::
::::

## Date and time of input to LLM

> 1700 27 June 2024

## Observations on the LLM Output

* The model is using a decorator `@dataclass` which automatically generates an init() method.
* a default rng is included
* The model uses a patient class, and creates patient instances for each of the patient types in the ASU, with a single generator function.
* It uses a run() function and a main() function, suggesting that it suspects modularity
* A follow-up prompt removed the decorator function. The patient class is retained for now, but the main function needs to be modified to change patient parameters.

## Sources reported by the LLM

None reported

## LLM Output

In [3]:
import simpy
import numpy as np
from dataclasses import dataclass, field
from typing import ClassVar

@dataclass
class PatientType:
    name: str
    interarrival_time: float
    count: int = 0
    rng: np.random.Generator = field(init=False)

    def __post_init__(self):
        self.rng = np.random.default_rng()

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

@dataclass
class ACUModel:
    env: simpy.Environment
    run_length: float = 5 * 365  # 5 years in days
    patient_types: ClassVar[dict] = {
        'Stroke': PatientType('Stroke', 1.2),
        'TIA': PatientType('TIA', 9.3),
        'Complex Neurological': PatientType('Complex Neurological', 3.6),
        'Other': PatientType('Other', 3.2)
    }
    total_arrivals: int = 0

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type: PatientType):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")
            print()

            # Patient immediately leaves the model
            # No additional processing needed

def main():
    env = simpy.Environment()
    model = ACUModel(env)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()


Time 0.05: Patient 0 (TIA) arrived
  Total arrivals: 1
  TIA arrivals: 1
  Next TIA arrival in 0.05 days

Time 0.51: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 1
  Next Stroke arrival in 0.51 days

Time 1.08: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 2
  Next Stroke arrival in 0.57 days

Time 1.32: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 3
  Next Stroke arrival in 0.24 days

Time 1.42: Patient 4 (Stroke) arrived
  Total arrivals: 5
  Stroke arrivals: 4
  Next Stroke arrival in 0.10 days

Time 1.48: Patient 5 (Stroke) arrived
  Total arrivals: 6
  Stroke arrivals: 5
  Next Stroke arrival in 0.06 days

Time 2.61: Patient 6 (Stroke) arrived
  Total arrivals: 7
  Stroke arrivals: 6
  Next Stroke arrival in 1.13 days

Time 3.17: Patient 7 (Complex Neurological) arrived
  Total arrivals: 8
  Complex Neurological arrivals: 1
  Next Complex Neurological arrival in 3.17 days

Time 4.54: Patient 8 (Stroke) arrived
  Total a

## Additional prompt: 
Modify the code so that it does not use the @dataclass decorator

### Response
In this modified version:

The PatientType class now has a regular __init__ method instead of using @dataclass. The count attribute is initialized to 0, and the rng (random number generator) is created in the initializer.
The ACUModel class is also converted to a regular class. The patient_types dictionary is now a class variable instead of a class variable with ClassVar annotation.
The __init__ method of ACUModel is explicitly defined, initializing the env, run_length, and total_arrivals attributes.
The field import from dataclasses and the typing import are no longer needed, so they've been removed.

This version of the code functions identically to the previous version but without using the @dataclass decorator. It's more verbose in the class definitions, but it gives you more explicit control over the class structure and initialization process.

In [5]:
import simpy
import numpy as np

class PatientType:
    def __init__(self, name, interarrival_time):
        self.name = name
        self.interarrival_time = interarrival_time
        self.count = 0
        self.rng = np.random.default_rng()

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

class ACUModel:
    patient_types = {
        'Stroke': PatientType('Stroke', 1.2),
        'TIA': PatientType('TIA', 9.3),
        'Complex Neurological': PatientType('Complex Neurological', 3.6),
        'Other': PatientType('Other', 3.2)
    }

    def __init__(self, env, run_length=5 * 365):  # 5 years in days
        self.env = env
        self.run_length = run_length
        self.total_arrivals = 0

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")
            print()

            # Patient immediately leaves the model
            # No additional processing needed

def main():
    env = simpy.Environment()
    model = ACUModel(env)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()


Time 0.36: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Next Stroke arrival in 0.36 days

Time 0.59: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Next Stroke arrival in 0.23 days

Time 1.39: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Next Stroke arrival in 0.80 days

Time 1.67: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Next Stroke arrival in 0.27 days

Time 3.47: Patient 4 (Stroke) arrived
  Total arrivals: 5
  Stroke arrivals: 5
  Next Stroke arrival in 1.81 days

Time 4.64: Patient 5 (Other) arrived
  Total arrivals: 6
  Other arrivals: 1
  Next Other arrival in 4.64 days

Time 7.24: Patient 6 (Stroke) arrived
  Total arrivals: 7
  Stroke arrivals: 6
  Next Stroke arrival in 3.77 days

Time 8.87: Patient 7 (Other) arrived
  Total arrivals: 8
  Other arrivals: 2
  Next Other arrival in 4.23 days

Time 9.46: Patient 8 (Stroke) arrived
  Total arrivals: 9
  Stroke arrivals: 7
  Next 

## Additional prompt: 
Modify the main function to accept different patient parameters

### Response:

To test running the model with new patient parameters, you can modify the ACUModel class to accept custom patient parameters. Here's how you can do that:

Modify the ACUModel class to accept a dictionary of patient types and their parameters. 
Modify the main() function to allow for testing different parameters. 
Now you can run the model with different parameters by calling main() with custom patient parameters.

In [6]:
import simpy
import numpy as np

class PatientType:
    def __init__(self, name, interarrival_time):
        self.name = name
        self.interarrival_time = interarrival_time
        self.count = 0
        self.rng = np.random.default_rng()

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

class ACUModel:
    def __init__(self, env, patient_params=None, run_length=5 * 365):
        self.env = env
        self.run_length = run_length
        self.total_arrivals = 0
        
        if patient_params is None:
            self.patient_types = {
                'Stroke': PatientType('Stroke', 1.2),
                'TIA': PatientType('TIA', 9.3),
                'Complex Neurological': PatientType('Complex Neurological', 3.6),
                'Other': PatientType('Other', 3.2)
            }
        else:
            self.patient_types = {
                name: PatientType(name, interarrival_time)
                for name, interarrival_time in patient_params.items()
            }

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")
            print()

            # Patient immediately leaves the model
            # No additional processing needed

def main(patient_params=None, run_length=5*365):
    env = simpy.Environment()
    model = ACUModel(env, patient_params, run_length)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()



Time 1.82: Patient 0 (Complex Neurological) arrived
  Total arrivals: 1
  Complex Neurological arrivals: 1
  Next Complex Neurological arrival in 1.82 days

Time 1.83: Patient 1 (Complex Neurological) arrived
  Total arrivals: 2
  Complex Neurological arrivals: 2
  Next Complex Neurological arrival in 0.02 days

Time 2.17: Patient 2 (TIA) arrived
  Total arrivals: 3
  TIA arrivals: 1
  Next TIA arrival in 2.17 days

Time 2.19: Patient 3 (Other) arrived
  Total arrivals: 4
  Other arrivals: 1
  Next Other arrival in 2.19 days

Time 2.20: Patient 4 (TIA) arrived
  Total arrivals: 5
  TIA arrivals: 2
  Next TIA arrival in 0.03 days

Time 2.62: Patient 5 (Stroke) arrived
  Total arrivals: 6
  Stroke arrivals: 1
  Next Stroke arrival in 2.62 days

Time 2.81: Patient 6 (Other) arrived
  Total arrivals: 7
  Other arrivals: 2
  Next Other arrival in 0.62 days

Time 3.63: Patient 7 (Stroke) arrived
  Total arrivals: 8
  Stroke arrivals: 2
  Next Stroke arrival in 1.01 days

Time 4.71: Patient 8

## Observation:
This works, and allows scenarios to be set up but as it is taking a different approach it might make prompt testing challenging later.

## Testing

* **Extreme value test 1**: TIA, Compelex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

In [12]:
M = 10_000_000

### Extreme value test 1

* The code is modified so that all patient types have a mean inter-arrival rate of $M$ except stroke

**Result: PASS**

In [15]:
def test_1(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': 1.2,
            'TIA': M,
            'Complex Neurological': M,
            'Other': M
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [16]:
test_1(M)

Time 1.42: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Next Stroke arrival in 1.42 days

Time 2.19: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Next Stroke arrival in 0.78 days

Time 2.27: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Next Stroke arrival in 0.08 days

Time 3.09: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Next Stroke arrival in 0.82 days

Time 4.70: Patient 4 (Stroke) arrived
  Total arrivals: 5
  Stroke arrivals: 5
  Next Stroke arrival in 1.60 days

Time 4.70: Patient 5 (Stroke) arrived
  Total arrivals: 6
  Stroke arrivals: 6
  Next Stroke arrival in 0.00 days

Time 5.72: Patient 6 (Stroke) arrived
  Total arrivals: 7
  Stroke arrivals: 7
  Next Stroke arrival in 1.02 days

Time 5.73: Patient 7 (Stroke) arrived
  Total arrivals: 8
  Stroke arrivals: 8
  Next Stroke arrival in 0.01 days

Time 6.22: Patient 8 (Stroke) arrived
  Total arrivals: 9
  Stroke arrivals: 9
 

### Extreme value test 2

* The code is again modified so that all patient type also have a mean inter-arrival rate of $M$

**Result: PASS**

In [17]:
def test_2(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': M,
            'TIA': M,
            'Complex Neurological': M,
            'Other': M
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [18]:
test_2(M)


Simulation completed.
Total simulation time: 730.00 days
Total patient arrivals: 0
Total Stroke arrivals: 0
Total TIA arrivals: 0
Total Complex Neurological arrivals: 0
Total Other arrivals: 0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

**Result: PASS**

In [21]:
TEN_DAYS = 10

In [24]:
def test_3(run_length):
    # Create the simulation environment
    if __name__ == "__main__":
        main(run_length=run_length)  # Run for 2 years with custom parameters


In [25]:
test_3(TEN_DAYS)

Time 0.23: Patient 0 (Complex Neurological) arrived
  Total arrivals: 1
  Complex Neurological arrivals: 1
  Next Complex Neurological arrival in 0.23 days

Time 0.34: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 1
  Next Stroke arrival in 0.34 days

Time 1.40: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 2
  Next Stroke arrival in 1.06 days

Time 1.57: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 3
  Next Stroke arrival in 0.17 days

Time 1.80: Patient 4 (Stroke) arrived
  Total arrivals: 5
  Stroke arrivals: 4
  Next Stroke arrival in 0.23 days

Time 1.82: Patient 5 (Stroke) arrived
  Total arrivals: 6
  Stroke arrivals: 5
  Next Stroke arrival in 0.02 days

Time 2.30: Patient 6 (TIA) arrived
  Total arrivals: 7
  TIA arrivals: 1
  Next TIA arrival in 2.30 days

Time 2.33: Patient 7 (Stroke) arrived
  Total arrivals: 8
  Stroke arrivals: 6
  Next Stroke arrival in 0.51 days

Time 3.51: Patient 8 (Stroke) arrived
  Total a